# Modelo de Ia (Gaussian)

In [1]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import  train_test_split,KFold,cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer,make_column_selector
from collections import Counter

In [2]:
df = pd.read_excel('../base/incluses_tratado.xlsx')

display(df.head(5))

,Participante da Comunidade LGBTQIA+,Faixa Etária,Identidade de Gênero,Orientação Sexual,Cidade/Estado,Escolaridade,Usa Apps para Oportunidades de Emprego?,Preferência de Cursos,Desafios de Emprego por Gênero,Interesse em Empreender,Situação no Mercado de Trabalho,Usa Redes Sociais?,Finalidade do Uso do App Incluses
0,Sim,Menos de 18 anos,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo ou cursando,Sim,Prefiro cursos on-line,Sim,Não,Empregado (a),Sim,Todos acima
1,Sim,Menos de 18 anos,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo ou cursando,Tenho interesse,"Não tenho preferência, pode ser on-line ou pre...",Sim,Não,Empregado (a),Sim,Não usaria o app
2,Sim,Menos de 18 anos,Mulher cisgênero,Pansexual,Região Norte,Ensino Fundamental completo ou cursando,Não,"Não tenho preferência, pode ser on-line ou pre...",Não,Talvez,Empregado (a),Sim,Fazer cursos de qualificação
3,Sim,Menos de 18 anos,Mulher cisgênero,Bissexual,Região Sudeste,Ensino Fundamental completo ou cursando,Não,"Não tenho preferência, pode ser on-line ou pre...",Sim,Não,Empregado (a),Sim,Não usaria o app
4,Sim,Menos de 18 anos,Homem transgênero,Prefiro não informar,Região Sudeste,Nenhum,Não,"Não tenho preferência, pode ser on-line ou pre...",Sim,Sim,Desempregado (a),Sim,Fazer cursos de qualificação


In [3]:
df['Finalidade do Uso do App Incluses'] = df['Finalidade do Uso do App Incluses'].replace(
    {"Encontrar oportunidades de emprego": "sim", 
     "Fazer cursos de qualificação": "sim", 
     "Divulgar conteúdos profissionais": "sim",
     'Divulgar conteúdos profissionais\xa0':"sim" ,
     "Todos acima": "sim", "Não usaria o app": "não"}
    )

In [4]:
df_resposta = df['Finalidade do Uso do App Incluses']
df_atributo = df.iloc[:, :-1]

#Resposta
label_encoder = LabelEncoder()

#Treino
preprocessador = ColumnTransformer(
    transformers=[
        ('ordinal_encoder', OrdinalEncoder(), make_column_selector(dtype_include=['object','bool'])),
    ],
    remainder='passthrough',
    verbose_feature_names_out= False 
)

In [5]:
# Pré-processamento dos atributos e da resposta
df_atributo = pd.DataFrame(preprocessador.fit_transform(df_atributo), columns=preprocessador.get_feature_names_out())
df_resposta = pd.DataFrame(label_encoder.fit_transform(df_resposta))

# Definindo os valores a serem testados
random_states = np.arange(0, 51, 1)  # Testando random_state de 0 a 50
k_neighbors_values = range(1, 21)  # Testando k_neighbors de 1 a 20
numfolds = 5
kf = KFold(n_splits=numfolds, shuffle=True, random_state=0)

best_accuracy = 0
best_random_state = None
best_k_neighbors = None

# Loop para testar diferentes combinações de random_state e k_neighbors no SMOTE
for random_state in random_states:
    for k_neighbors in k_neighbors_values:
        # Contar as amostras em cada classe
        class_counts = Counter(df_resposta[0])  # Ajuste para acessar a coluna correta de resposta
        min_class_samples = min(class_counts.values())
        
        # Garantir que k_neighbors não seja maior do que as amostras da classe minoritária
        if k_neighbors < min_class_samples:
            # Aplicando SMOTE com os valores atuais de random_state e k_neighbors
            smote = SMOTE(random_state=random_state, k_neighbors=k_neighbors)
            df_atributo_resampled, df_resposta_resampled = smote.fit_resample(df_atributo, df_resposta)

            # Criando e validando o modelo GaussianNB
            modelo_gaussian = GaussianNB()  # Modelo Naive Bayes Gaussiano
            validacaoCruzada = cross_val_score(modelo_gaussian, df_atributo_resampled, df_resposta_resampled, cv=kf)

            # Calculando a acurácia média
            mean_accuracy = validacaoCruzada.mean()

            # Verificando se a acurácia atual é a melhor
            if mean_accuracy > best_accuracy:
                best_accuracy = mean_accuracy
                best_random_state = random_state
                best_k_neighbors = k_neighbors

# Exibindo a melhor combinação de random_state e k_neighbors para SMOTE
print(f"Melhor random_state: {best_random_state}")
print(f"Melhor k_neighbors no SMOTE: {best_k_neighbors}")
print(f"Melhor acurácia média: {best_accuracy:.4f}")

# Aplicando SMOTE com os melhores parâmetros encontrados
smote = SMOTE(random_state=best_random_state, k_neighbors=best_k_neighbors)
df_atributo, df_resposta = smote.fit_resample(df_atributo, df_resposta)


c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-pack

Melhor random_state: 14
Melhor k_neighbors no SMOTE: 2
Melhor acurácia média: 0.8890


c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-pack

In [6]:
# df_atributo = pd.DataFrame(preprocessador.fit_transform(df_atributo), columns=preprocessador.get_feature_names_out())
# df_resposta = pd.DataFrame(label_encoder.fit_transform(df_resposta))


# smote = SMOTE(random_state=42,k_neighbors=2)
# df_atributo, df_resposta = smote.fit_resample(df_atributo, df_resposta)

In [7]:
test_sizes = np.arange(0.2, 0.3, 0.01)  # Testando test_size de 0.20 a 0.29
random_states = np.arange(1, 81, 1)  # Testando random_state de 1 a 80
n_splits_values = range(2, 11)  # Testando n_splits de 2 a 10 (valores comuns para K-Fold)

# Dicionário para armazenar os resultados
melhor_acuracia = 0
melhor_test_size = None
melhor_random_state = None
melhor_n_splits = None

# Loop para testar diferentes combinações de test_size, random_state e n_splits
for n_splits in n_splits_values:
    for test_size in test_sizes:
        for random_state in random_states:
            # Dividir os dados em treino e teste
            df_atributos_treino, df_atributos_teste, df_resposta_treino, df_resposta_teste = train_test_split(
                df_atributo, df_resposta.values, test_size=test_size, random_state=random_state
            )

            # Inicializando o classificador GaussianNB
            modelo_gaussian = GaussianNB()

            # Inicializando a validação cruzada com o valor atual de n_splits
            kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

            # Validando o modelo com validação cruzada
            validacaoCruzada = cross_val_score(modelo_gaussian, df_atributos_treino, df_resposta_treino.ravel(), cv=kf)

            # Calculando a acurácia média
            mean_accuracy = validacaoCruzada.mean()

            # Verificando se a acurácia é a melhor até agora
            if mean_accuracy > melhor_acuracia:
                melhor_acuracia = mean_accuracy
                melhor_test_size = test_size
                melhor_random_state = random_state
                melhor_n_splits = n_splits

# Exibindo a melhor combinação de test_size, random_state e n_splits
print(f"Melhor test_size: {melhor_test_size}")
print(f"Melhor random_state: {melhor_random_state}")
print(f"Melhor n_splits: {melhor_n_splits}")
print(f"Melhor acurácia média: {melhor_acuracia:.4f}")

# Aplicando a melhor divisão com os parâmetros otimizados
df_atributos_treino, df_atributos_teste, df_resposta_treino, df_resposta_teste = train_test_split(
    df_atributo, df_resposta.values, test_size=melhor_test_size, random_state=melhor_random_state
)

# # Aplicar SMOTE no conjunto de treino
# smote = SMOTE(random_state=24, k_neighbors=2)  # Ajuste os parâmetros conforme necessário
# atributos_treino, resposta_treino = smote.fit_resample(df_atributos_treino, df_resposta_treino)

# Convertendo para arrays, se necessário
atributos_teste = df_atributos_teste.values
resposta_teste = df_resposta_teste

Melhor test_size: 0.25000000000000006
Melhor random_state: 70
Melhor n_splits: 9
Melhor acurácia média: 0.9206


In [8]:
classificador_bayes = GaussianNB()
modelo = classificador_bayes.fit(df_atributos_treino, df_resposta_treino)

probabilidades = classificador_bayes.predict_proba(atributos_teste)
classificacao = classificador_bayes.predict(atributos_teste)

c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [9]:
# print("Probabilidades das classes para os dados de teste:\n", probabilidades)
# print("Classificação dos dados de teste:\n", classificacao)

taxa_de_acerto = accuracy_score(resposta_teste, classificacao)
print("Taxa de acerto", taxa_de_acerto)
print("\nRelatório de Classificação:\n", classification_report(resposta_teste, classificacao))
print("Matriz de Confusão:\n", confusion_matrix(resposta_teste, classificacao))

Taxa de acerto 0.8095238095238095

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.91      0.77      0.83        13
           1       0.70      0.88      0.78         8

    accuracy                           0.81        21
   macro avg       0.80      0.82      0.81        21
weighted avg       0.83      0.81      0.81        21

Matriz de Confusão:
 [[10  3]
 [ 1  7]]
